<a href="https://colab.research.google.com/github/jyporse/YOLO/blob/main/Pok%C3%A9mon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Goolgle Drive에 저장된 cuDNN 파일 가져오기

In [2]:
import os
if not os.path.exists('usr/local/cuda/include/cudnn.h'):
    !tar -xzvf drive/MyDrive/Kobot/YOLO/darknet/cuDNN/cudnn-11.1-linux-x64-v8.0.5.39.tgz -C /usr/local/
    !chmod a+r /usr/local/cuda/include/cudnn.h

!cat /usr/local/cuda/include/cudnn.h

cuda/include/cudnn.h
cuda/include/cudnn_adv_infer.h
cuda/include/cudnn_adv_train.h
cuda/include/cudnn_backend.h
cuda/include/cudnn_cnn_infer.h
cuda/include/cudnn_cnn_train.h
cuda/include/cudnn_ops_infer.h
cuda/include/cudnn_ops_train.h
cuda/include/cudnn_version.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.8
cuda/lib64/libcudnn.so.8.0.5
cuda/lib64/libcudnn_adv_infer.so
cuda/lib64/libcudnn_adv_infer.so.8
cuda/lib64/libcudnn_adv_infer.so.8.0.5
cuda/lib64/libcudnn_adv_train.so
cuda/lib64/libcudnn_adv_train.so.8
cuda/lib64/libcudnn_adv_train.so.8.0.5
cuda/lib64/libcudnn_cnn_infer.so
cuda/lib64/libcudnn_cnn_infer.so.8
cuda/lib64/libcudnn_cnn_infer.so.8.0.5
cuda/lib64/libcudnn_cnn_train.so
cuda/lib64/libcudnn_cnn_train.so.8
cuda/lib64/libcudnn_cnn_train.so.8.0.5
cuda/lib64/libcudnn_ops_infer.so
cuda/lib64/libcudnn_ops_infer.so.8
cuda/lib64/libcudnn_ops_infer.so.8.0.5
cuda/lib64/libcudnn_ops_train.so
cuda/lib64/libcudnn_ops_train.so.8
cuda/lib64/libcudnn_o

### Google Drive에 저장된 Darknet, Weights, cfg, data, custom , backup 파일 로컬로 가져오기


In [4]:
import os
if not os.path.exists('darknet'):
    os.makedirs('darknet')
%cd darknet
%ls
!ls -la '/content/drive/MyDrive/Kobot/YOLO/darknet/bin/darknet'
!cp -r /content/drive/MyDrive/Kobot/YOLO/darknet/bin/darknet ./darknet
!chmod +x ./darknet
#YOLO data 가져오기
!cp -r /content/drive/MyDrive/Kobot/YOLO/darknet/weights .
!cp -r /content/drive/MyDrive/Kobot/YOLO/darknet/cfg .
!cp -ar /content/drive/MyDrive/Kobot/YOLO/darknet/data .
!cp -r /content/drive/MyDrive/Kobot/darknet/Images/ .
# Custom data가 있는 폴더 복사
#!cp -r /content/Gdrive/MyDrive/Kobot/darknet/backup .
%ls

/content/darknet
-rw------- 1 root root 6845664 Jan 27 09:05 /content/drive/MyDrive/Kobot/YOLO/darknet/bin/darknet
cfg/  darknet*  data/  Images/  weights/


### 이미지를 업로드, 다운로드 또는 보여주기 함수

In [5]:
#download files

def imShow(path):
    import cv2
    import matplotlib.pyplot as plt
    %matplotlib inline

    img = cv2.imread(path)
    height , width = img.shape[:2]
    resized_img = cv2.resize(img, (3*width, 3*height),interpolation = cv2.INTER_CUBIC)

    fig = plt.gcf()
    fig.set_size_inches(18,10)
    plt.axis("off")
    #plt.rcParams['figure.figsize'] = [10,5]
    plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
    plt.show()

def upload():
    from google.colab import files
    uploaded = files.upload()
    for name, data in uploaded.items():
        with open(name, "wb") as f:
            f.write(data)
            print("saved file", name)
def download(path):
    from google.colab import files
    files.download(path)

### 라벨링한 데이터들의 cfg , test, train, names 파일을 하나하나 입력하는것은 너무 힘든일이다 따라서 라벨링한 이미지와 데이터를 변환해주는 코드

In [8]:
%cd /content/darknet

/content/darknet


In [9]:
%ls

backup/  cfg/  darknet*  data/  Images/  weights/


In [10]:
import os

current_path = os.path.abspath(os.curdir)
#print(current_path) 현재 경로 확인 
COLAB_DARKNET_PATH = '/content/drive/MyDrive/Kobot/darknet'
# Darknet 경로

YOLO_COLAB_IMAGE_PATH = current_path + '/Images/'
YOLO_COLAB_FORMAT_PATH = current_path + '/Images/'
# Darknet 학습할 이미지가 있는 경로

class_count = 0 # 클래스 개수
test_percentage  = 0.2 #test 비율
paths = []

with open(YOLO_COLAB_FORMAT_PATH + '/' + 'classes.names', 'w') as names, \
     open(YOLO_COLAB_FORMAT_PATH + '/' + 'classes.txt', 'r') as txt:
     for line in txt:
         names.write(line)
         #print(line)
         class_count += 1
     print("[classes.names] is created")
 # 로컬 데이터에서 라벨링 된 이미지들의 calsses.txt를 읽어 확장명 names로 바꿔주고 class 개수를 추가해주는 함수

with open(YOLO_COLAB_FORMAT_PATH + '/' + 'custom_data.data', 'w') as data:
    data.write('classes = ' + str(class_count) + '\n')
    data.write('train = ' + YOLO_COLAB_IMAGE_PATH  + 'train.txt' + '\n')
    data.write('test = ' + YOLO_COLAB_IMAGE_PATH  + 'test.txt' + '\n')
    data.write('names = ' + YOLO_COLAB_IMAGE_PATH + 'classes.names' + '\n')
    data.write('backup = backup')
    print("[custom_data.data] is created")
# 로컬 데이터에 custom_data.data 포멧의 입력정보를 입력하는 코드

os.chdir(YOLO_COLAB_IMAGE_PATH)
for current_dir, dirs, files in os.walk('.'):
    for f in files:
        if (f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png') ):
            image_path = current_path + '/Images/' + f
            paths.append(image_path + '\n')

## 로컬 데이터에 저장된  images 폴더를 순회하면서 확장자가 jpg, jpeg, png 라면 경롤

paths_test = paths[:int(len(paths) * test_percentage)]

paths = paths[int(len(paths) * test_percentage) :]

with open(YOLO_COLAB_FORMAT_PATH + '/' + 'train.txt', 'w') as train_txt:
    for path in paths :
        train_txt.write(path)
    print("[train_txt] is created")

with open(YOLO_COLAB_FORMAT_PATH + '/' + 'test.txt', 'w') as test_txt:
    for path in paths :
        test_txt.write(path)
    print("[test_txt] is created")    
    


[classes.names] is created
[custom_data.data] is created
[train_txt] is created
[test_txt] is created


In [11]:
%cd /content/darknet

/content/darknet


### XML파일을 YOLO format 으로 변환해주는 코드도 있지만 여기서는 다루지 않음

### 학습 시작 

학습시간이 엄청 오래걸림
darknet53.conv.74 파일은 초기 학습 데이터
custom.data.data 의 경로는 구글 드라이브 경로여야 한다(아니여도 상관없음).

In [ ]:
!./darknet detector train Images/custom_data.data Images/custom-train-yolo.cfg weights/darknet53.conv.74 -dont_show 

In [7]:
if os.path.exists('./backup'):
    !cp -r ./backup "/content/drive/MyDrive/Kobot/darknet"

if not os.path.exists('./backup'):
    os.makedirs('./backup')
    !cp -r "/content/drive/MyDrive/Kobot/darknet/backup/custom-train-yolo_last _700.weights" ./backup

### 100단위로 저장된다 왜 ?




In [12]:
!./darknet detector map Images/custom_data.data Images/custom-train-yolo.cfg backup/custom-train-yolo_last _700.weights

CUDA status Error: file: ./src/dark_cuda.c : () : line: 38 : build time: Jan 27 2022 - 09:04:25 

 CUDA Error: no CUDA-capable device is detected
Darknet error location: ./src/dark_cuda.c, check_error, line #69
CUDA Error: no CUDA-capable device is detected: Bad file descriptor
